## Importing Required Packages

In [1]:
import os
import numpy as np
import pandas as pd
import statistics
import rioxarray
import xarray as xr
import rasterio
import statistics
from tqdm import tqdm

## Function to convert data to annualy

In [2]:
def aggregate_to_annual(data_array):
    """
    Aggregate monthly data to annual values.
    """
    return data_array.resample(time='A').mean(dim='time')

## Function to calculate max and min value per cell
Also also applying first filter

In [3]:
def calculate_max_min_per_cell(data_array):
    
    # Calculate the maximum and minimum values excluding outliers
    
    # mean_val = data_array.mean(dim='time')
    # std_val = data_array.std(dim='time')
    
    # Exclude outliers within each grid cell
    # valid_data = data_array.where((data_array >= (mean_val - 3 * std_val)) & (data_array <= (mean_val + 3 * std_val)), other=np.nan)
    
    valid_data = data_array
    
    # Calculate the maximum and minimum values for each cell after excluding outliers
    max_val_per_cell = valid_data.max(dim='time')
    min_val_per_cell = valid_data.min(dim='time')
    
    return max_val_per_cell, min_val_per_cell

## Load Data

In [4]:
# Define paths to your data
data_dir = "../Processed_Data/HTI_Indices_Resampled/"
koppen_map_path = "../Raw_Data/koppen_geiger_0p5.tif"
output_path = "../Processed Data/Output of analysis"

# Load Koppen map
koppen_data = rioxarray.open_rasterio(koppen_map_path).squeeze()
regions = np.unique(koppen_data)

# Models and indices
models = os.listdir(data_dir)
scenarios = ['Historical','SSP1-2.6','SSP2-4.5','SSP5-8.5']
indices = ["AT_in", "AT_out", "DI", "ET", "HI", "HMI", "MDI", "NET", "SAT", "sWGBT", "WBT", "WCT"]  # Replace with actual indices names

max_min_results = {region: {index: {model: {scenario: {"max": None, "min": None, "max_cross_consi": None, "min_cross_consi": None, "max_cross_years": [], "min_cross_years": []} for scenario in scenarios} for model in models} for index in indices}  for region in regions}

## Finding Max and Min Values of Indices on historical Period (1850-2014)

In [5]:
for model in models:
    print(f'Precssing of {model} is started')
    model_dir = os.path.join(data_dir, model)
    model_dir_historical = os.path.join(data_dir, model, 'Historical')
    
    for index in tqdm(indices,"processing Indices"):
        
        index_file = os.path.join(model_dir_historical, f'resampled_{index}.nc')
        data_array_historical = xr.open_dataset(index_file)[index]
        data_array_annual_historical = aggregate_to_annual(data_array_historical)
        
        max_val, min_val = calculate_max_min_per_cell(data_array_annual_historical)  # got historical max and min value per cell of index
        
        for region in regions:
            region_mask = (koppen_data == region).compute()
            
            region_max_val_per_cell = max_val.where(region_mask, other = np.nan)
            region_min_val_per_cell = min_val.where(region_mask, other = np.nan)
            
            #region_mean_of_max_val_per_cell = region_max_val_per_cell.mean()
            #region_mean_of_min_val_per_cell = region_min_val_per_cell.mean()
            
            #region_std_of_max_val_per_cell = region_max_val_per_cell.std()
            #region_std_of_min_val_per_cell = region_min_val_per_cell.std()
          
            #Valid_max_val_for_region = region_max_val_per_cell.where((region_max_val_per_cell >= (region_mean_of_max_val_per_cell-3*region_std_of_max_val_per_cell)) & (region_max_val_per_cell <= (region_mean_of_max_val_per_cell + 3*region_std_of_max_val_per_cell)) , other = np.nan)
            #Valid_min_val_for_region = region_min_val_per_cell.where((region_min_val_per_cell >= (region_mean_of_min_val_per_cell-3*region_std_of_min_val_per_cell)) & (region_min_val_per_cell <= (region_mean_of_min_val_per_cell + 3*region_std_of_min_val_per_cell)) , other = np.nan)
            
            Valid_max_val_for_region = region_max_val_per_cell
            Valid_min_val_for_region = region_min_val_per_cell
            
            max_val_of_index_for_this_region = Valid_max_val_for_region.max()
            min_val_of_index_for_this_region = Valid_min_val_for_region.min()
            
            max_min_results[region][index][model]['Historical']["max"] = max_val_of_index_for_this_region
            max_min_results[region][index][model]['Historical']["min"] = min_val_of_index_for_this_region
        
    print(f'Processing of {model} is Done\n')

Precssing of ACCESS-CM2 (Australia) is started


processing Indices: 100%|██████████████████████████████████████████████████████████████| 12/12 [04:38<00:00, 23.21s/it]


Processing of ACCESS-CM2 (Australia) is Done

Precssing of ACCESS-ESM1-5 (Australia) is started


processing Indices: 100%|██████████████████████████████████████████████████████████████| 12/12 [05:29<00:00, 27.46s/it]


Processing of ACCESS-ESM1-5 (Australia) is Done

Precssing of CanESM5 (Canada) is started


...

Processing of TaiESM1 is Done

Precssing of UKESM1-0-LL is started


processing Indices: 100%|██████████████████████████████████████████████████████████████| 12/12 [04:37<00:00, 23.11s/it]


Processing of UKESM1-0-LL is Done

Precssing of CESM2-WACCM (USA) is started


processing Indices: 100%|██████████████████████████████████████████████████████████████| 12/12 [05:33<00:00, 27.81s/it]

Processing of CESM2-WACCM (USA) is Done



## Finding Exposure Timing

In [6]:
future_scenarios = ['SSP1-2.6','SSP2-4.5','SSP5-8.5']

for model in models:
    print(f'Precssing of {model} is started')
    model_dir = os.path.join(data_dir, model)
    
    for scenario in future_scenarios:
        scenario_dir = os.path.join(model_dir,scenario)

        for index in tqdm(indices,f"processing {model} {scenario} {index}"):
            index_file_of_scenario = os.path.join(scenario_dir,f'resampled_{index}.nc')
            data_array_scenario = xr.open_dataset(index_file_of_scenario)[index]
            data_array_annual_scenario = aggregate_to_annual(data_array_scenario)
            
            #filtering######################
            # mean_val = data_array_annual_scenario.mean(dim='time')
            # std_val = data_array_annual_scenario.std(dim='time')

            # Exclude outliers within each grid cell
            # valid_data = data_array_annual_scenario.where((data_array_annual_scenario >= (mean_val - 3 * std_val)) & (data_array_annual_scenario <= (mean_val + 3 * std_val)), other=np.nan)
            
            valid_data = data_array_annual_scenario
            data_array_annual_scenario = valid_data
            ################################
            
            filtered_scenario_data = data_array_annual_scenario  # for now assuming filter is applied

            for region in regions:

                region_mask = (koppen_data == region).compute()
                
                region_data = filtered_scenario_data.where(region_mask, other = np.nan)

                max_cross_year = [] # years when index value will be higher than its historical max
                min_cross_year = [] # years when index value will be lower than its historical min

                max_value_for_region_per_year = region_data.max(dim='x').max(dim='y')
                min_value_for_region_per_year = region_data.min(dim='x').min(dim='y')
                
                for i in range(85):
                    if max_value_for_region_per_year[i] > max_min_results[region][index][model]['Historical']["max"]:
                        max_cross_year.append(2015+i)
                    if min_value_for_region_per_year[i] < max_min_results[region][index][model]['Historical']["min"]:
                        min_cross_year.append(2015+i)
                
                max_min_results[region][index][model][scenario]["max_cross_years"] = max_cross_year
                max_min_results[region][index][model][scenario]["min_cross_years"] = min_cross_year
                
                max_count = 1
                max_consi_year = None
                
                for i in range(len(max_cross_year)):
                    if(i==0):
                        continue
                    else:
                        if(max_cross_year[i] == max_cross_year[i-1]+1):
                            if(max_consi_year == None):
                                max_consi_year = max_cross_year[i-1]
                            max_count= max_count+1
                            if(max_count>=5):
                                break
                        else:
                            if(max_count>=5):
                                break
                            else:
                                max_count=1;
                                max_consi_year = None
                                
                    if (i == (len(max_cross_year)-1)) & (max_count<5):
                        max_consi_year = None;

                min_count = 1
                min_consi_year= None
                
                for j in range(len(min_cross_year)):
                    if(j == 0):
                        continue
                    else:
                        if(min_cross_year[j] == min_cross_year[j-1]+1):
                            if(min_consi_year == None):
                                min_consi_year = min_cross_year[j-1]
                            min_count= min_count+1
                            if(min_count>=5):
                                break
                        else:
                            if(min_count>=5):
                                break
                            else:
                                min_count=1;
                                min_consi_year = None
                    if (j == (len(max_cross_year)-1)) & (max_count<5):
                        max_consi_year = None;
                
                max_min_results[region][index][model][scenario]["max_cross_consi"] = max_consi_year
                max_min_results[region][index][model][scenario]["min_cross_consi"] = min_consi_year
                

Precssing of ACCESS-CM2 (Australia) is started


processing ACCESS-CM2 (Australia) SSP5-8.5 WCT: 100%|██████████████████████████████████| 12/12 [03:57<00:00, 19.78s/it]


Precssing of ACCESS-ESM1-5 (Australia) is started


processing ACCESS-ESM1-5 (Australia) SSP1-2.6 WCT: 100%|███████████████████████████████| 12/12 [04:20<00:00, 21.71s/it]
processing ACCESS-ESM1-5 (Australia) SSP2-4.5 WCT: 100%|███████████████████████████████| 12/12 [04:51<00:00, 24.25s/it]
processing ACCESS-ESM1-5 (Australia) SSP5-8.5 WCT: 100%|███████████████████████████████| 12/12 [04:32<00:00, 22.67s/it]


Precssing of CanESM5 (Canada) is started


...

processing TaiESM1 SSP5-8.5 WCT: 100%|█████████████████████████████████████████████████| 12/12 [03:52<00:00, 19.35s/it]


Precssing of UKESM1-0-LL is started


processing UKESM1-0-LL SSP5-8.5 WCT: 100%|█████████████████████████████████████████████| 12/12 [04:04<00:00, 20.38s/it]


Precssing of CESM2-WACCM (USA) is started


processing CESM2-WACCM (USA) SSP5-8.5 WCT: 100%|███████████████████████████████████████| 12/12 [04:02<00:00, 20.24s/it]


## Summary of Max & min values in history and exposure time of future of different models

In [7]:
summary_results = []

for region in regions:
    for index in indices:
        for model in models:
            
            #numpy_array = max_min_results[region][model][index]["max"].to_numpy()
            max_vals = max_min_results[region][index][model]['Historical']["max"].values
            min_vals = max_min_results[region][index][model]['Historical']["min"].values

            max_ = np.nanmedian(max_vals)
            min_ = np.nanmedian(min_vals)

            summary_results.append({
                "Region": region,
                "Index": index,
                "Model": model,
                "Historical Max": max_,
                "Historical Min": min_,

                "SSP1-2.6 Max_Cross_start_5_consicutive_years": max_min_results[region][index][model]['SSP1-2.6']["max_cross_consi"],
                "SSP1-2.6Min_Cross_start_5_consicutive_years": max_min_results[region][index][model]['SSP1-2.6']["min_cross_consi"],
                "SSP1-2.6Max_Cross_Years": max_min_results[region][index][model]['SSP1-2.6']["max_cross_years"],
                "SSP1-2.6Min_Cross_Years": max_min_results[region][index][model]['SSP1-2.6']["min_cross_years"],

                "SSP2-4.5 Max_Cross_start_5_consicutive_years": max_min_results[region][index][model]['SSP2-4.5']["max_cross_consi"],
                "SSP2-4.5 Min_Cross_start_5_consicutive_years": max_min_results[region][index][model]['SSP2-4.5']["min_cross_consi"],
                "SSP2-4.5 Max_Cross_Years": max_min_results[region][index][model]['SSP2-4.5']["max_cross_years"],
                "SSP2-4.5 Min_Cross_Years": max_min_results[region][index][model]['SSP2-4.5']["min_cross_years"],

                "SSP5-8.5 Max_Cross_start_5_consicutive_years": max_min_results[region][index][model]['SSP5-8.5']["max_cross_consi"],
                "SSP5-8.5 Min_Cross_start_5_consicutive_years": max_min_results[region][index][model]['SSP5-8.5']["min_cross_consi"],
                "SSP5-8.5 Max_Cross_Years": max_min_results[region][index][model]['SSP5-8.5']["max_cross_years"],
                "SSP5-8.5 Min_Cross_Years": max_min_results[region][index][model]['SSP5-8.5']["min_cross_years"],
            })

summary_df = pd.DataFrame(summary_results)

In [17]:
summary_df.to_excel('../Processed_Data/Output of analysis/Non-styled-complete-data.xlsx')

Styling the dataframe

In [22]:
styled_df = summary_df.style.set_properties(**{'text-align': 'center'})
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
styled_df = styled_df.set_table_styles([dict(selector='td', props=[('text-align', 'right')])])
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('font-weight', 'bold')])])
styled_df = styled_df.set_table_styles([dict(selector='td', props=[('font-weight', 'normal')])])
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('background-color', '#f2f2f2')])])
styled_df = styled_df.set_table_styles([dict(selector='td', props=[('background-color', '#e6e6e6')])])
styled_df = styled_df.set_table_styles([dict(selector='th', props=[('border-bottom', '1px solid #ddd')])])
styled_df = styled_df.set_table_styles([dict(selector='td', props=[('border-bottom', '1px solid #ddd')])])
styled_df = styled_df.set_caption('Abrupt Values')
styled_df = styled_df.format({ "Region": "{:.0f}",  "Max": "{:.2f}", "Min": "{:.2f}" })
styled_df = styled_df.hide(axis='index')
styled_df = styled_df.background_gradient(cmap='coolwarm')

#styled_df = styled_df.bar(subset=['Max', 'Min'])

# To display the styled DataFrame
# styled_df

In [29]:
styled_df.to_html('../Processed_Data/Output of analysis/complete_data.html')

In [23]:
styled_df.to_excel('../Processed_Data/Output of analysis/complete_data.xlsx')

In [24]:
summary_df.to_csv('../Processed_Data/Output of analysis/Non-styled-complete_data.csv')

## Finding one value of time median

In [19]:
final_results = {region: {index: {scenario: {"median_max_cross_consi": None, "median_min_cross_consi": None} for scenario in scenarios} for index in indices}  for region in regions}

for region in regions:
    for index in indices:
        
        max_min_list = {scenario: {'max_list' : [], 'min_list' : []} for scenario in scenarios}
        
        for scenario in scenarios:
            for model in models:
                if max_min_results[region][index][model][scenario]['max_cross_consi']:
                    max_min_list[scenario]['max_list'].append(max_min_results[region][index][model][scenario]['max_cross_consi'])
                if max_min_results[region][index][model][scenario]['min_cross_consi']:
                    max_min_list[scenario]['min_list'].append(max_min_results[region][index][model][scenario]['min_cross_consi'])
        
            if len(max_min_list[scenario]['max_list'])!=0:
                final_results[region][index][scenario]['median_max_cross_consi'] = statistics.median(max_min_list[scenario]['max_list'])

            if len(max_min_list[scenario]['min_list'])!=0:
                final_results[region][index][scenario]['median_min_cross_consi'] = statistics.median(max_min_list[scenario]['min_list'])

In [20]:
summary_median= []

for region in regions:
    for index in indices:
                
        summary_median.append({
            "Region": region,
            "Index": index,

            "Max_Median_SSP1-2.6": final_results[region][index]['SSP1-2.6']['median_max_cross_consi'],
            "Min_Median_SSP1-2.6": final_results[region][index]['SSP1-2.6']['median_min_cross_consi'],

            "Max_Median_SSP2-4.5": final_results[region][index]['SSP2-4.5']['median_max_cross_consi'],
            "Min_Median_SSP2-4.5": final_results[region][index]['SSP2-4.5']['median_min_cross_consi'],

            "Max_Median_SSP5-8.5": final_results[region][index]['SSP5-8.5']['median_max_cross_consi'],
            "Min_Median_SSP5-8.5": final_results[region][index]['SSP5-8.5']['median_min_cross_consi']
        })


summary_median_df = pd.DataFrame(summary_median)

In [25]:
styled_median_df = summary_median_df.style.set_properties(**{'text-align': 'center'})
styled_median_df = styled_median_df.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='td', props=[('text-align', 'right')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='th', props=[('font-weight', 'bold')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='td', props=[('font-weight', 'normal')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='th', props=[('background-color', '#f2f2f2')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='td', props=[('background-color', '#e6e6e6')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='th', props=[('border-bottom', '1px solid #ddd')])])
styled_median_df = styled_median_df.set_table_styles([dict(selector='td', props=[('border-bottom', '1px solid #ddd')])])
styled_median_df = styled_median_df.set_caption('Abrupt Values')
styled_median_df = styled_median_df.format({ "Region": "{:.0f}",  "Max_Median": "{:.2f}", "Min_Median": "{:.2f}" })
styled_median_df = styled_median_df.hide(axis='index')
styled_median_df = styled_median_df.background_gradient(cmap='coolwarm')

#styled_df = styled_df.bar(subset=['Max_Median', 'Min_Median'])

# To display the styled DataFrame
# styled_median_df

In [26]:
styled_median_df.to_excel('../Processed_Data/Output of analysis/median_values_of_exposure_timing-final-summary.xlsx')

In [27]:
summary_median_df.to_csv('../Processed_Data/Output of analysis/median_values_of_exposure_timing-final-summary.csv')

In [28]:
styled_median_df.to_html('../Processed_Data/Output of analysis/median_values_of_exposure_timing-final-summary.html')